In [ ]:
import tqdm
import numpy as np
import torch
import pandas as pd
import pickle
import time
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.set_option('mode.chained_assignment',  None)

import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import random
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader


In [ ]:
loss_dir = './loss'
model_dir = './model_save'
table_loc_master = "/data/Storage_DAS02/jaeyoon/230101_BreastCancer_Project/data/230629_BC_new/"
table_data_master = "230913_BT_Timeseries_new.xlsx"
table_data_2_master = "230913_EMR_Timeseries_new.xlsx"
Table_BT = pd.read_excel(table_loc_master+table_data_master)
Table_EMR = pd.read_excel(table_loc_master+table_data_2_master)

In [ ]:
Table_BT[var].values

In [ ]:
Table_EMR = pd.read_excel(table_loc_master+table_data_2_master)


In [ ]:
Var_bi = []
Var_con = []
for var in Table_EMR.columns[3:]:
    try:
        if len(np.unique(Table_EMR[var].values)) < 10:
            print(var)
            for val in np.unique(Table_EMR[var].values)[:-1]:
                print(val)
                Table_EMR[f"{var}_{val}"] = (Table_EMR[var] == val)
                Var_bi += [f"{var}_{val}"]
        else:
            Var_con += [var]
    except:
        continue

In [ ]:
for var in Table_BT.columns[3:]:
    Var_con += [var]
    if len(np.unique(Table_BT[var].values)) < 10:
        print(var, (np.unique(Table_BT[var].values)))

In [ ]:
Table_BT.to_excel('BT_subbbb.xlsx')

In [ ]:
past_ID = ''
for i in range(len(Table_BT)):
    if type(Table_BT.loc[i].new_ID) != str:
        Table_BT.loc[i,"new_ID"] = past_ID
    else:
        past_ID = Table_BT.loc[i].new_ID
Table_BT

In [ ]:
Table_EMR["count_idx"] = [0 for _ in range(len(Table_EMR))]
Table_EMR.index = [Table_EMR["new_ID"],Table_EMR["count_idx"]]
Table_EMR

In [ ]:
Table_BT["Date_gap"] = ['' for _ in range(len(Table_BT))]
Table_BT.index = [Table_BT["new_ID"],Table_BT["count_idx"]]
Table_BT

In [ ]:
for ID in Table_BT.index.get_level_values(0).unique().values:
    if ID in Table_EMR.index.get_level_values(0):
        Table_BT.loc[ID,"Date_gap"] = [int(((datetime.datetime.strptime(i, '%Y-%m-%d').timestamp()-datetime.datetime.strptime(str(Table_EMR.loc[ID]["Date"].values[0])[:10], '%Y-%m-%d').timestamp())/86400+30)//90)  for i in Table_BT.loc[ID]["Date"].values]
        

In [ ]:
Table_BT["count_idx"] = Table_BT["Date_gap"].values
Table_BT.index = [Table_BT["new_ID"],Table_BT["count_idx"]]
Table_BT

In [ ]:
Table_BT

In [ ]:
Var_bi

In [ ]:
Var_con

In [ ]:
Table_cat.to_excel('230913_concat_table.xlsx')

In [ ]:
pd.concat([Table_EMR,Table_BT]).sort_index(level = 0).iloc[:20]

In [ ]:
Table_cat = pd.concat([Table_BT,Table_EMR])
Table_cat = Table_cat.sort_index(level = 1).sort_index(level = 0)
Table_cat = Table_cat.iloc[2:] 
Table_cat = Table_cat.groupby(level = 0,axis = 1).first()
Table_cat

In [ ]:
Table_Date = np.array([i.timestamp() for i in Table_cat["Date"]])
Table_Date = (Table_Date-np.mean(Table_Date))/ np.std(Table_Date)
Table_Date = pd.DataFrame(Table_Date, index = Table_cat.index)
Table_Date

In [ ]:
np.log2(Table_cat+1)

In [ ]:
Table_cat.astype(float)

In [ ]:
Table_cat = Table_cat[Var_con + Var_bi]
Table_cat = np.log2(Table_cat.astype(float)+1)
for var in Var_con:
    Table_cat[var] = (Table_cat[var]-Table_cat[var].dropna().mean())/Table_cat[var].dropna().std()
    # Table_X[var] = Table_X[var] - Table_X[var].dropna().min() + eps
# Table_L_Con = Table_X[Var_Con].max(axis = 0)

Table_X_norm = Table_cat.copy()
Table_X_mask = Table_cat.isna()
Table_X_norm[Var_con] = Table_X_norm[Var_con].fillna(0)#(eps)
Table_X_norm[Var_bi] = Table_X_norm[Var_bi].fillna(0)
Table_val_Weight = (Table_X_mask.shape[0] / (Table_X_mask.shape[0]-Table_X_mask.sum(axis = 0)))

T_X_bi = Table_X_norm[Var_bi]
T_X_con = Table_X_norm[Var_con]
T_M_bi = Table_X_mask[Var_bi]
T_M_con = Table_X_mask[Var_con]
T_W_bi = Table_val_Weight[Var_bi]
T_W_con = Table_val_Weight[Var_con]
T_Date = Table_Date

In [ ]:
# output : ID_train & ID_test
ID_all = Table_cat.index.get_level_values(level=0).unique().values
np.random.shuffle(ID_all)

Train_ratio = 0.8
ID_train = ID_all[:int(len(ID_all)*Train_ratio)]
ID_test = ID_all[int(len(ID_all)*Train_ratio):]
print(ID_all.shape,ID_train.shape, ID_test.shape)

In [ ]:
# Save IDs
pd.DataFrame(ID_train).to_excel("Train_ID_set.xlsx")
pd.DataFrame(ID_test).to_excel("Test_ID_set.xlsx")

In [ ]:
# Dataset
class VAE_Dataset(Dataset):
    def __init__(self, X1, X2, M1, M2, T_Date, ID_list):

        self.x1 = torch.from_numpy(X1.loc[ID_list].values)
        self.m1 = torch.from_numpy(M1.loc[ID_list].values)
        
        self.x2 = torch.from_numpy(X2.loc[ID_list].values)
        self.m2 = torch.from_numpy(M2.loc[ID_list].values)

        self.op_data = torch.from_numpy(T_Date.loc[ID_list].values)

    def __len__(self):
        return len(self.x1)

    def __getitem__(self, idx):
        return self.x1[idx], self.x2[idx], self.m1[idx], self.m2[idx], self.op_data[idx]
    
class VAE_sampler():
    def __init__(self, rand_p = 0.05):
        self.rand_p = rand_p
    def VAE_rand_fn(self, batch):
        x1,x2,m1,m2,op_data = zip(*batch)
        r_mask_ratio = self.rand_p
        eps = 1e-6
        
        x1 = torch.stack(x1).float()
        x2 = torch.stack(x2).float()
        m1 = torch.stack(m1).bool()
        m2 = torch.stack(m2).bool()
        op_data = torch.stack(op_data).float()
        
        
        r1 = (torch.rand_like(x1)<r_mask_ratio).bool()
        x1 = x1 * (~r1) + 0 * r1
        m1 = m1 + r1
        
        r2 = (torch.rand_like(x2)<r_mask_ratio).bool()
        x2 = x2 * (~r2) + 0 * r2
        m2 = m2 + r2
        
        return x1, x2, m1, m2, op_data

In [ ]:
VAE_sampler_use = VAE_sampler(0.1)
train_dataloader = DataLoader(VAE_Dataset(T_X_bi,T_X_con,T_M_bi,T_M_con,T_Date,ID_train),
                              batch_size=64, shuffle=True, collate_fn=VAE_sampler_use.VAE_rand_fn)
test_dataloader = DataLoader(VAE_Dataset(T_X_bi,T_X_con,T_M_bi,T_M_con,T_Date,ID_test),
                             batch_size=64, shuffle=True, collate_fn=VAE_sampler_use.VAE_rand_fn)

In [ ]:
class BC_VAE(nn.Module):
    def __init__(self,
                 input_sizes = [0,0],
                 layer_size = [[],[]],
                 norm_mth = 'Layer_norm'):
        
        super(BC_VAE,self).__init__()
        input_size = input_sizes[0] + input_sizes[1]
        Data_model_layer = layer_size
        Data_model_layer[0][0] = input_size*2+1
        Data_model_layer[1][0] += input_size+1
        Data_model_layer[1][-1] = input_size

        self.Encoder = Test_Encoder(Data_model_layer[0], normalize = norm_mth)
        self.Decoder = Test_Decoder(Data_model_layer[1], normalize = norm_mth, output_sizes = input_sizes)
        self.Data_model_layer = Data_model_layer
        
    def forward(self,inputs): # x,mask,y
        x1,x2,m1,m2,op_date = inputs
        mask_with_date = torch.concat([m1,m2,op_date+torch.randn_like(op_date)*0.5],axis = -1)
        x = torch.concat([x1,x2], axis= -1)
        m_s, log_v_s, sample_s = self.Encoder(x,mask_with_date)
        outputs = self.Decoder(sample_s,mask_with_date)
        return outputs, m_s, log_v_s
    
class Test_Encoder(nn.Module):
    def __init__(self, layer_size = [32,128,24,16], normalize = False):
        super(Test_Encoder,self).__init__()
        self.layer_size = layer_size
        self.Encoder = VAE_Encoder_masklayer(layer_size, normalize = normalize)
        
    def forward(self,x,mask):
        return self.Encoder(torch.concat([x,mask],axis = -1))
    
class Test_Decoder(nn.Module):
    def __init__(self, layer_size = [32,128,24,16], normalize = False, output_sizes = []):
        super(Test_Decoder,self).__init__()
        self.output_sizes = output_sizes
        self.layer_size = layer_size
        
        self.Merged_Decoder = Merged_Decoder(layer_size, normalize = normalize,output_sizes = output_sizes)
    def forward(self,sampled,mask):
        return self.Merged_Decoder(torch.concat([sampled,mask],axis = -1))
    
class VAE_Encoder_masklayer(nn.Module):
    def __init__(self, layer_size = [32,128,24,16], normalize = False):
        super(VAE_Encoder_masklayer,self).__init__()
        self.layer_size = layer_size
        self.depth = len(layer_size)-1
        self.blocks = nn.ModuleDict()
        for n in range(self.depth-1):
            self.blocks.__setitem__(str(n),FLblock(layer_size[n], layer_size[n+1], normalize = normalize, drop_out = 0.1))
        n = self.depth-1
        self.m_block = FLblock(layer_size[n], layer_size[n+1], activ_layer = False, normalize = False)
        self.v_block = FLblock(layer_size[n], layer_size[n+1], activ_layer = False, normalize = False)
    def forward(self,x):
        for n in range(self.depth-1):
            x = self.blocks[str(n)](x)
        x_m = self.m_block(x)
        x_log_v = self.v_block(x)
        return x_m, x_log_v, torch.randn_like(x_m).mul(torch.exp(0.5*x_log_v)).add_(x_m)

class Merged_Decoder(nn.Module):
    def __init__(self, layer_size = [32,128,24,16], normalize = False, drop_out = 0.1, output_sizes = [0,0]):
        super().__init__()
        self.layer_size = layer_size
        self.depth = len(layer_size)-1
        self.blocks = nn.ModuleDict()
        for n in range(self.depth-1):
            self.blocks.__setitem__(str(n),FLblock(layer_size[n], layer_size[n+1], normalize = normalize, drop_out = drop_out))#, bias = False))
        n = self.depth-1
        # bi 
        self.Bi_p_block = FLblock(layer_size[n], output_sizes[0], activ_layer = 'sigmoid', normalize = False, clamp = 1e4)
        # con : all GGM
        self.Con_mse_block = FLblock(layer_size[n], output_sizes[1], activ_layer = False, normalize = False, clamp = True)
        
    def forward(self,x):
        for n in range(self.depth-1):
            x = self.blocks[str(n)](x)
        output = {}
        output['Continuous'] = (self.Con_mse_block(x))                
        output['Binary'] = (self.Bi_p_block(x))

        return output
    
class FLblock(nn.Module):
    def __init__(self, channel_in,channel_out, activ_layer = 'relu',drop_out = False, normalize = False, clamp = False, bias = True):
        super().__init__()
        self.activ_layer = activ_layer
        self.Linear_in = nn.Linear(channel_in,channel_out, bias=bias)
        self.drop_out = drop_out
        self.normalize = normalize
        self.clamp = clamp
        if normalize == 'Layer_norm':
            self.Norm = nn.LayerNorm([channel_out])
        elif normalize == 'Batch_norm':
           self.Norm = nn.BatchNorm1d([channel_out])
    def forward(self, x):
        block_1 = self.Linear_in(x)
        if self.clamp is not False:
            if self.clamp is True:
                block_1 = torch.clamp(block_1,max = 1e6, min = -1e6)
            else:
                block_1 = torch.clamp(block_1,max = self.clamp, min = -1*self.clamp)
                
        if self.activ_layer is not False: # False, Relu, Sigmoid, Exp
            if self.activ_layer == 'sigmoid':
                block_1 = nn.Sigmoid()(block_1)
            elif self.activ_layer == 'exp':
                block_1 = torch.exp(block_1)
            elif self.activ_layer == 'softmax':
                block_1 = nn.Softmax(dim = -1)(block_1)
            elif self.activ_layer == 'relu':
                block_1 = nn.ReLU()(block_1)
            elif self.activ_layer == 'softplus':
                block_1 = nn.Softplus()(block_1)
        if self.normalize is not False:
            block_1 = self.Norm(block_1)
        if self.drop_out is not False:
            block_1 = nn.Dropout(p = self.drop_out)(block_1)
        return block_1

In [ ]:
def Run_model(model,optimizer,dataloader,device,train=True):
    if train:
        model.train()
    else:
        model.eval()
        
    epoch_loss = {
        "KLD" : 0,
        "Continuous" : 0,
        "Binary" : 0,
    }
    i_num = 0
    for x1,x2,m1,m2,op_date in dataloader:
        with torch.autograd.detect_anomaly():
            i_num += x1.shape[0]
            # model.zero_grad()
            optimizer.zero_grad()
            x1 = x1.to(device)
            x2 = x2.to(device)
            m1 = m1.to(device)
            m2 = m2.to(device)
            op_date = op_date.to(device)
            Model_out_s, m_s, log_v_s = model([x1,x2,m1,m2,op_date])
            
            loss_KLD = -0.5 * torch.sum(1 + log_v_s - m_s.pow(2) - log_v_s.exp(), axis = -1).mean()
            
            # GGM
            dist = 'Continuous'
            Cont_y = Model_out_s[dist]
            Cont_x = x2
            Cont_mask = m2
            Cont_Weight = T_W_con        
            case_Cont = (-1)*(Cont_x-Cont_y).pow(2)
                
            loss_Continuous = (-1)*torch.sum(torch.mul(torch.sum(torch.mul((Cont_mask<0.1).type(torch.float64),case_Cont),axis = 0)
                                            /x2.shape[0],Cont_Weight))
            
            # Ber
            dist = 'Binary'
            
            Ber_p = Model_out_s[dist]
                
            Ber_x = x1
            Ber_mask = m1
            Ber_Weight = T_W_bi
            
            case_Ber = (torch.mul((Ber_x<0.5).float(),torch.log((1-Ber_p)*0.9+0.05))
                        +torch.mul((Ber_x>=0.5).float(),torch.log(Ber_p)*0.9+0.05))
                
            loss_Binary = (-1)*torch.sum(torch.mul(torch.sum(torch.mul((Ber_mask<0.1).float(),case_Ber),axis = 0)
                                            /x1.shape[0],Ber_Weight))
            
            # loss = (loss_Continuous*10 + loss_Binary)*5
            loss = loss_KLD + (loss_Continuous + loss_Binary)*5
            epoch_loss["KLD"] += loss_KLD * x1.shape[0]
            epoch_loss["Continuous"] += loss_Continuous * x1.shape[0]
            epoch_loss["Binary"] += loss_Binary * x1.shape[0]
            if train:
                try:
                    loss.backward()
                    optimizer.step()
                except:
                    print(Ber_x,case_Ber)
    epoch_loss["KLD"] /= i_num
    epoch_loss["Continuous"] /= i_num
    epoch_loss["Binary"] /= i_num
    return epoch_loss

In [ ]:
latent_count = 100
layer_data = [[1, 1024, 512, latent_count], [latent_count, 512, 1024, 1]]
device = "cuda:1"
model = BC_VAE(input_sizes = [T_X_bi.shape[1],T_X_con.shape[1]],
                 layer_size = layer_data,
                 norm_mth = 'Layer_norm').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)


In [ ]:
T_W_con = torch.tensor(T_W_con.values, device = device).requires_grad_(False)
T_W_bi = torch.tensor(T_W_bi.values, device = device).requires_grad_(False)
T_W_con = (T_W_con+1).log2()
# T_W_con[:10] = 4*T_W_con[:10]
T_W_bi = (T_W_bi+1).log2()

In [ ]:
epochs = 50
timer_use = tqdm.tqdm(range(epochs))
losses = []

for epoch in timer_use:
    loss = Run_model(model,optimizer,train_dataloader,device,True)

    t = f"epoch: {epoch:04d} " + \
        f"loss: KL {loss['KLD']:.3f} C {loss['Continuous']:.3f} B {loss['Binary']:.3f} |"
        
    loss = Run_model(model,optimizer,test_dataloader,device,False)

    t += f"validate : KL {loss['KLD']:.3f} C {loss['Continuous']:.3f} B {loss['Binary']:.3f}"
        
    timer_use.set_description(t)

    
    losses.append(loss)
torch.save(model, f"{model_dir}/230913_VAE_beta_5.pth")


In [ ]:
rep_1 = 2
sampled = np.array([])
sampled_real = np.array([])
sampled_mask = np.array([])
m_ss, log_v_ss = [], []

for x1,x2,m1,m2,op_date in test_dataloader:
    model.zero_grad()
    x1 = x1.to(device)
    x2 = x2.to(device)
    m1 = m1.to(device)
    m2 = m2.to(device)
    op_date = op_date.to(device)
    for _ in range(rep_1):
        dist = 'Continuous'
        Model_out_s, m_s, log_v_s = model([x1,x2,m1,m2,op_date])
        GGM_x =  x2.detach().cpu().numpy()
        GGM_mask = m2.detach().cpu().numpy()
        data_sample = Model_out_s[dist].detach().cpu().numpy()
        sampled_real = np.concatenate([sampled_real,GGM_x],axis = 0) if len(sampled_real) else GGM_x
        sampled = np.concatenate([sampled,data_sample],axis = 0) if len(sampled) else data_sample
        sampled_mask = np.concatenate([sampled_mask,GGM_mask],axis = 0) if len(sampled_mask) else GGM_mask
        m_ss = np.concatenate([m_ss,m_s.detach().cpu().numpy()],axis = 0) if len(m_ss) else m_s.detach().cpu().numpy()
        log_v_ss = np.concatenate([log_v_ss,log_v_s.detach().cpu().numpy()],axis = 0) if len(log_v_ss) else log_v_s.detach().cpu().numpy()
    
    # # Ber
    # dist = 'Binary'
    
    # Ber_p = Model_out_s[dist]
        
    # Ber_x = x1
    # Ber_mask = m1

In [ ]:
idx_i = 0
plt.figure(figsize=(5,5))
for n, i in enumerate(Var_con):
    idx_i +=1
    plt.subplot(5,5,idx_i)
    plt.scatter(sampled_real[sampled_mask[:,n] == 0,n],sampled[sampled_mask[:,n] == 0,n],s = 1, c = 'k')
    plt.xticks([0,np.max(sampled_real[sampled_mask[:,n] == 0,n])],['',''])
    plt.yticks([0,np.max(sampled_real[sampled_mask[:,n] == 0,n])],['',''])
    plt.xlim(np.min(sampled_real[sampled_mask[:,n] == 0,n])-0.5,np.max(sampled_real[sampled_mask[:,n] == 0,n]+0.5))
    plt.ylim(np.min(sampled_real[sampled_mask[:,n] == 0,n])-0.5,np.max(sampled_real[sampled_mask[:,n] == 0,n]+0.5))
    # plt.title(i)
    if idx_i == 25:
        plt.show()
        plt.figure(figsize=(5,5))
        idx_i = 0
plt.show()
        

In [ ]:
plt.figure(figsize=(5,5))
idx_i = 0
for i in range(100):
    idx_i+=1
    plt.subplot(5,5,idx_i)
    plt.scatter(m_ss[:,i],np.exp(0.5*log_v_ss[:,i]), s = 1)
    plt.plot([-3,3],[0,0],'k')
    plt.plot([-3,3],[1,1],'k-')
    plt.plot([0,0],[-1,3],'k')
    plt.xticks([-2,0,2])
    plt.yticks([0,0.5,1])
    plt.xlim(-3,3)
    plt.ylim(-0.1,1.51)
    if idx_i == 25:
        plt.show()
        plt.figure(figsize=(5,5))
        idx_i = 0

In [ ]:
Table_1

In [ ]:
Table_sub

In [ ]:
Table_X_norm.loc["P_001469"]

In [ ]:
Table_cat.index = [Table_cat.index.get_level_values(0),Table_cat.index.get_level_values(1)//4]
Table_cat = Table_cat.groupby(level = [0,1]).first()

In [ ]:
Table_cat = Table_cat.groupby(level = [0,1]).first()

In [ ]:

Table_X_norm.index = [Table_X_norm.index.get_level_values(0),Table_X_norm.index.get_level_values(1)//4]
Table_X_norm = Table_X_norm.groupby(level = [0,1]).first()
Table_X_norm

In [ ]:

Table_X_mask.index = [Table_X_mask.index.get_level_values(0),Table_X_mask.index.get_level_values(1)//4]
Table_X_mask = Table_X_mask.groupby(level = [0,1]).first()
Table_X_mask

In [ ]:
Table_Date.index = [Table_Date.index.get_level_values(0),Table_Date.index.get_level_values(1)//4]
Table_Date = Table_Date.groupby(level = [0,1]).first()
Table_Date

In [ ]:
Table_in_norm

In [ ]:
Idx_con = [n for n,i in enumerate(Table_cat.columns) if i in Var_con]
Idx_bi = [n for n,i in enumerate(Table_cat.columns) if i in Var_bi]

In [ ]:
VAE_Encoder = model.Encoder
ID_set = {}
ID_set['train'] = ID_train
ID_set['test'] = ID_test
IDs = np.unique(Table_cat.index.get_level_values(level = 0))
ID_list = []
ID_year_list = []
TF_in_data = np.array([])
TF_in_std = np.array([])
TF_in_mask = np.array([])
tqdm_epoch = tqdm.tqdm(IDs)
using_z = [i for i in range(100)]
for ID in tqdm_epoch:
    TF_in_data_sub = torch.zeros((1,10,len(using_z)), device = device).float()
    TF_in_std_sub = torch.ones((1,10,len(using_z)), device = device).float()
    TF_in_mask_sub = torch.ones((1,10,1), device = device).float()
    Table_ID_idx = np.where(Table_cat.index.get_level_values(level = 0) == ID)[0]
    Table_sub = Table_cat.loc[ID]
    Table_in_date = torch.tensor(Table_sub["Test_date_normed"].loc[Table_sub.index < 9].values,device = device).float().unsqueeze(1)
    ID_year = []
    ID_idx  = []
    for n, i in enumerate(Table_sub.index):
        if i < 9: # (-1 ~ 9year의 데이터만 사용할 예정)
            ID_year += [i]
            ID_idx += [Table_ID_idx[n]]
    if len(ID_year)>0:
        ID_year_sub= []
        Table_in_norm = torch.tensor(np.array(Table_X_norm.loc[ID].loc[ID_year])[:,Idx_bi+Idx_con],device = device).float()
        Table_in_mask = torch.tensor(np.array(Table_X_mask.loc[ID].loc[ID_year])[:,Idx_bi+Idx_con],device = device).float()
        Table_in_mask = torch.concat([Table_in_mask,Table_in_date+torch.randn_like(Table_in_date)*0.5],axis = -1)
        m_s, log_v_s, sample_s = VAE_Encoder(Table_in_norm,Table_in_mask)
        for i in range(m_s.shape[0]):
            if ID_year[i] <-1:
                if -1 in ID_year_sub:
                    continue
                else:
                    ID_year[i] = -1
                    ID_year_sub += [-1]
            else:
                ID_year_sub += [ID_year[i]]
            TF_in_data_sub[0,(ID_year[i]+1),using_z] = m_s[i,using_z]
            TF_in_std_sub[0,(ID_year[i]+1),using_z] = (log_v_s[i,using_z]/2).exp()
            TF_in_mask_sub[0,(ID_year[i]+1)] = 0
        TF_in_data = TF_in_data_sub if TF_in_data.shape[0] ==0 else torch.concat([TF_in_data,TF_in_data_sub],axis = 0)
        TF_in_std = TF_in_std_sub if TF_in_std.shape[0] ==0 else torch.concat([TF_in_std,TF_in_std_sub],axis = 0)
        TF_in_mask = TF_in_mask_sub if TF_in_mask.shape[0] ==0 else torch.concat([TF_in_mask,TF_in_mask_sub],axis = 0)
        ID_list += [ID]
        ID_year_list += [ID_year_sub]


In [ ]:
ID_year_list

In [ ]:
Table_in_mask

In [ ]:
Table_X_norm[Var_bi]

In [ ]:
with open('TF_in_data_all.pickle', 'wb') as f:
    pickle.dump([TF_in_data, TF_in_std, TF_in_mask, ID_list, ID_year_list], f, pickle.HIGHEST_PROTOCOL)

In [ ]:
Table_X_norm.loc[[ID]].iloc[Table_X_norm.loc[[ID]].index.get_level_values(level = 1)<9]